### 1.1 Tensorflow low api介绍
1. Tensorflow low api分为两个部分  
  1. 构建计算图:`tf.Graph`
  2. 运行计算图:`tf.Session`
2. 何为计算图 
  1. 计算图是由一系列Tensorflow操作组成的Graph.其中顶点为`Tensorflow operation`, 边为输入输出的张量
  2. Tensorflow的张量类为`tf.Tensors`
3. 如下, 构建第一个Tensorflow demo

In [1]:
# 为了pthon2兼容python3声明的特性包, python3下不用导入, 这些特性已默认包含在python3中
from __future__ import division        # 精确除: int/int=float
from __future__ import absolute_import # 绝对路径导入
from __future__ import print_function  # 使用print函数, 而不是print关键字
import tensorflow as tf

/devkit/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
x = tf.constant([[1], [2], [3], [4]],dtype=tf.float32)
y = tf.constant([[0], [-1], [-2], [-3]],dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x) # 相当于keras functional api
loss = tf.losses.mean_squared_error(labels=y,predictions=y_pred) # 损失函数
optimazer = tf.train.RMSPropOptimizer(0.01) # 优化器

train = optimazer.minimize(loss) # 使用优化器最小化损失函数

init = tf.global_variables_initializer() # 初始化layers中的参数

sess = tf.Session()
sess.run(init)
for i in range(1000): #训练1000次
    x = sess.run((train,loss))  # session.run执行一次tensor运算
    if i == 999:
        print('loss:',x[1]) 
print(sess.run(y_pred))

### 1.2 Tensor

#### 一. Tensor简介
1. 使用`tf.Tensor`对象表示张量. 确定维度的张量也会产生确定维度的张量, 但有些张量的维度只在计算图运行时出现
2. `tf.Tensor`对象的rank用来区分标量,向量,矩阵,n维矩阵

#### 二. 不用session.run即可获取结果的操作
1. 声明不同rank的tensor
2. tensor index切片
3. 获取tensor的shape  
 返回`tf.Tensor.Shape`
4. tensorflow初始化  
 `tf.zeros(Tensor.shape)`

In [6]:
# rank = 0
tf.Variable(99,dtype=tf.int16)
# rank = 1
mystr = tf.Variable(['Hello'],tf.string)
col_numbers = tf.Variable([3.14,6.28],dtype=tf.float32)
col_1 = col_numbers[1]
# rank = n
myxor = tf.Variable([[False, True],[True, False]], tf.bool)
matrix1 = tf.Variable([[111,222],[333,444]],dtype=tf.int32)
zeros = tf.zeros(matrix1.shape)

#### 三. 获取张量维度
`tf.rank()`

In [7]:
n = tf.rank(myxor)

In [20]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print (sess.run(mystr))
    print (sess.run(n))
    print (sess.run(col_1))
    res = sess.run(col_numbers)
    print (sess.run(zeros))

[b'Hello']
2
6.28
[[0. 0.]
 [0. 0.]]


#### 四. 获取与tensor相同内容的ndarray
1. 使用`Tensor.eval()`  
2. `Tensor.eval()`需要在tf.Session开启的状况下  
3, 使用`tf.placeholder`声明的变量, 由于没有确定值, 所以无法在脱离计算图的环境下使用`eval`  
 此时需要指定参数`tf.eval(feed_dict={})`

In [16]:
with tf.Session() as sess : 
    constant = tf.constant([1,2,3])
    tensor1 = constant * constant
    print(tensor1.eval())

[1 4 9]


In [26]:
# eval指定feed_dict
p = tf.placeholder(tf.float32)
t = p+1.0
with tf.Session() as sess : 
#     print(t.eval()) #报错
    res = t.eval(feed_dict={p:2.0})
    print(res)

3.0


#### 五. 关于打印tensor
1. 如下形式只能打印出`tf.Tensor`对象, 此时这个对象还没有值  
  ```python
t = <<some tensorflow operation>>
print(t)
  ```

In [30]:
t = tf.Variable([1,2,3],dtype=tf.int32)
t = tf.Print(t,[t])
t+1

<tf.Tensor 'add_6:0' shape=(3,) dtype=int32>

In [35]:
x=tf.constant([2,3,4,5])
x=tf.Print(x,[x,x.shape,'any thing i want'],message='Debug message:',summarize=100)
 
with tf.Session() as sess:
    print(sess.run(x))

[2 3 4 5]
